In [147]:
pip install transformers

In [149]:
import os
import time
import transformers as trn
import numpy as np
import pandas as pd
import tensorflow as tf 
import pickle
from transformers.trainer_utils import set_seed

print(tf.config.list_physical_devices('GPU'))

random_seed=4321

set_seed(random_seed)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [150]:
with open('/content/drive/MyDrive/CSE573-SWM-Data/AaplLabelledNewsData-OrderPreserved.pkl', 'rb') as f:
    labelled_aapl_news_df = pickle.load(f)
    
with open('/content/drive/MyDrive/CSE573-SWM-Data/AmznLabelledNewsData-OrderPreserved.pkl', 'rb') as f:
    labelled_amzn_news_df = pickle.load(f)

In [ ]:
print(labelled_aapl_news_df.head(10))
print(labelled_amzn_news_df.head(10))

       news_timestamp  ... label
0 2017-12-07 20:00:00  ...    -1
1 2017-12-08 21:37:00  ...     1
2 2017-12-12 01:55:00  ...     1
3 2017-12-12 22:10:00  ...     1
4 2017-12-14 11:42:00  ...     1
5 2017-12-14 12:31:00  ...     1
6 2017-12-15 02:32:00  ...     1
7 2017-12-15 12:04:00  ...     1
8 2017-12-18 20:05:00  ...    -1
9 2017-12-21 00:00:00  ...     1

[10 rows x 5 columns]
       news_timestamp  ... label
0 2017-12-15 12:04:00  ...     1
1 2017-12-21 12:04:00  ...    -1
2 2017-12-21 12:27:00  ...    -1
3 2017-12-22 00:00:00  ...     1
4 2017-12-22 20:57:00  ...     1
5 2017-12-27 00:00:00  ...    -1
6 2017-12-27 11:43:00  ...    -1
7 2017-12-28 00:00:00  ...    -1
8 2017-12-28 11:39:00  ...    -1
9 2017-12-28 11:39:00  ...    -1

[10 rows x 5 columns]


In [151]:
columns = ['merged_tokens']
corpus_aapl = []
corpus_amzn = []

#can try to vectorize this
for index, row in labelled_aapl_news_df.iterrows():
    corpus_aapl.append(' '.join(row['tokens']))
    if index % 10000 == 0:
        print("Completed %d rows for apple" % index)
    
for index, row in labelled_amzn_news_df.iterrows():
    corpus_amzn.append(' '.join(row['tokens']))
    if index % 5000 == 0:
        print("Completed %d rows for amazon" % index)

Completed 0 rows for apple
Completed 10000 rows for apple
Completed 20000 rows for apple
Completed 30000 rows for apple
Completed 40000 rows for apple
Completed 50000 rows for apple
Completed 60000 rows for apple
Completed 70000 rows for apple
Completed 0 rows for amazon
Completed 5000 rows for amazon
Completed 10000 rows for amazon
Completed 15000 rows for amazon


In [152]:
aapl_news = pd.DataFrame(corpus_aapl, columns = ['sentences'])
amzn_news = pd.DataFrame(corpus_amzn, columns = ['sentences'])

In [153]:
amzn_news['labels'] = labelled_amzn_news_df['label']

In [154]:
aapl_news['labels'] = labelled_aapl_news_df['label']

In [155]:
print(aapl_news)

                                               sentences labels
0      nape summit also annual luncheon tom speaker n...     -1
1      huge x comeback tour follow note big year best...      1
2      browser os ad server make first move competito...      1
3                humanitarian place conflict free region      1
4                                          prefer invest      1
...                                                  ...    ...
71936                                         deem store      1
71937                deem use key develop tool day later      1
71938  tweet trader share trade tuesday stock track h...      1
71939  obtain intern memo tri spin led block defend u...      1
71940  correct stock predict post test learn past yea...      1

[71941 rows x 2 columns]


In [ ]:
max(aapl_news['sentences'].str.len())

2816

In [ ]:
aapl_news.sentences.str.len().max()

2816

In [ ]:
pip install transformers

     |████████████████████████████████| 2.0MB 17.0MB/s 
     |████████████████████████████████| 3.2MB 56.9MB/s 
     |████████████████████████████████| 870kB 51.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=e41823c88e00afb10b47cea6b02f02c91a1cb9807bf238ead9246705ce719055
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [156]:
from transformers import BertTokenizer, TFBertModel

In [157]:
tokenizer_class = {}
model_class = {}

In [158]:
tokenizer_class['finbert'] = BertTokenizer.from_pretrained('/content/drive/MyDrive/data/fin_vocab')

In [159]:
aapl_news['sentences'][1:10]

1    huge x comeback tour follow note big year best...
2    browser os ad server make first move competito...
3              humanitarian place conflict free region
4                                        prefer invest
5    invest good news fund new job refurbish close ...
6    park made new invest four transact cisco partn...
7    amazon soon sell pull e platform two year ago ...
8    said pandora p sirius cumulus player radio aud...
9    read earn north america may start gain speed s...
Name: sentences, dtype: object

In [ ]:
aapl_sample = aapl_news.head()

In [ ]:
print(aapl_sample)

                                           sentences labels
0  nape summit also annual luncheon tom speaker n...     -1
1  huge x comeback tour follow note big year best...      1
2  browser os ad server make first move competito...      1
3            humanitarian place conflict free region      1
4                                      prefer invest      1


In [162]:
encoded_ids_list = []
encoded_tokens_list = []

In [163]:
for text in aapl_news['sentences']:
    encoded_ids = tokenizer_class['finbert'](text, return_tensors='tf')
    encoded_ids_list.append(encoded_ids)
    #print(encode_ids_list)
    encoded_tokens = tokenizer_class['finbert'].convert_ids_to_tokens(encoded_ids['input_ids'].numpy()[0])
    #print(type(encoded_tokens))
    encoded_tokens_list.append(encoded_tokens)

In [164]:
len(encoded_ids_list)

71941

In [165]:
encoded_ids_df = pd.DataFrame(encoded_ids_list)
encoded_ids_df

,attention_mask,input_ids,token_type_ids
0,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
1,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
2,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
3,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
4,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
...,...,...,...
71936,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
71937,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
71938,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."
71939,"((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(3, shape=(), dtype=int32), tf.Tens...","((tf.Tensor(0, shape=(), dtype=int32), tf.Tens..."


In [166]:
encoded_ids_df.to_pickle('/content/drive/MyDrive/encoded_ids_fin.pkl')

In [167]:
model_class['finbert'] = TFBertModel.from_pretrained('/content/drive/MyDrive/models/finbert', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [168]:
def get_model_output(text, model, tokenizer):
    """ Generate the model output for a given text """
    # [CLS] is automatically added by the tokenizer
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(encoded_input)
    return output

In [ ]:
0:1001 ---> 1001 --> f
1001: 4003 --> 3002 --> features_two
4003:7004 --> 3001 --> features_three
7004:10005 --> 3001 --> features_four
10005:13006 --> 3001 --> features_five
13006:18007 --> 5001 --> features_six
18007:25001 --> 6994    --> features_sev
25001:35002 --> 10001 --> features_eight
35002:45003 --> 10001 --> features_nine
45003:55004 --> 10001 --> features_ten
55004:65005 --> 10001 --> features_eleven
65005:71941 -->       --> features_twelve


In [297]:
output_list = []
print(output_list)

[]


In [298]:
for text in aapl_news['sentences'][65005:71941]:
    output = get_model_output(text, model_class['finbert'], tokenizer_class['finbert'])
    output_list.append(output)
    
# Checking various attributes of the final output
print(output.pooler_output.shape)
#print(output.last_hidden_state.shape)
#print(output.attentions.shape)
print(len(output_list))
#for i in output_list:
#    print(i.pooler_output.shape) 

(1, 768)
6936


In [ ]:
print(len(output_list))

1001


In [ ]:
features = pd.DataFrame(output_list)

In [ ]:
print(features)

                                      last_hidden_state  ... attentions
0     (((tf.Tensor(0.84315795, shape=(), dtype=float...  ...       None
1     (((tf.Tensor(0.2291574, shape=(), dtype=float3...  ...       None
2     (((tf.Tensor(-0.025799029, shape=(), dtype=flo...  ...       None
3     (((tf.Tensor(-0.1316198, shape=(), dtype=float...  ...       None
4     (((tf.Tensor(-0.09950562, shape=(), dtype=floa...  ...       None
...                                                 ...  ...        ...
996   (((tf.Tensor(0.2838955, shape=(), dtype=float3...  ...       None
997   (((tf.Tensor(0.4969926, shape=(), dtype=float3...  ...       None
998   (((tf.Tensor(0.1430857, shape=(), dtype=float3...  ...       None
999   (((tf.Tensor(-0.06504179, shape=(), dtype=floa...  ...       None
1000  (((tf.Tensor(1.1982353, shape=(), dtype=float3...  ...       None

[1001 rows x 4 columns]


In [309]:
pooler_output_list = []

In [310]:
for items in output_list:
  pooler_output_list.append(items.pooler_output.numpy())

#print(pooler_output_list[1])
len(pooler_output_list)
#numpy_ar = output_list[0].pooler_output.numpy()


6936

In [ ]:
pooler_output_list[1].shape

(1, 768)

In [311]:
features_tw = pd.DataFrame()

In [312]:
features_tw

""


In [313]:
for items in pooler_output_list:
  features_tw = features_tw.append(pd.DataFrame(items), ignore_index= True)



In [314]:
features_tw

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.992519,0.162391,-0.999795,0.706420,0.933451,0.401218,-0.961831,-0.149847,0.999060,-0.232698,0.121431,0.863439,-0.999933,-0.007186,-0.043103,0.047057,-0.150552,-0.008027,-0.299268,-0.968755,0.221582,0.993981,0.999986,0.932122,0.562211,0.999990,0.995873,-0.048188,0.066478,0.047830,0.214891,0.355612,0.109028,-0.324649,-0.077918,0.999925,-0.226519,-0.147049,0.123917,0.179057,...,0.210639,0.035975,-0.999829,-0.144361,-0.154900,0.037330,0.152969,0.047440,0.304120,0.078779,0.231826,0.117461,-0.063812,0.424873,0.999985,-0.982603,-0.177264,0.177069,0.038038,-0.019516,-0.997425,-0.998444,-0.394916,0.792626,-0.055630,-0.076407,0.998449,-0.999859,0.980598,-0.240181,0.157679,-0.114261,-0.945299,0.154400,0.330171,-0.104870,-0.993522,0.876691,0.962474,-0.020555
1,0.998427,0.223063,-0.995576,0.886130,0.655030,0.606384,-0.960571,-0.009474,0.998343,-0.194857,0.125635,0.983134,-0.999014,-0.023436,-0.227324,0.070896,-0.109216,-0.025605,-0.083169,-0.984666,-0.009526,0.998770,0.999993,0.979310,-0.078207,0.999999,0.997064,0.126618,0.230820,0.124092,0.039584,0.295741,-0.231607,-0.269251,0.106118,0.999997,-0.141759,-0.008067,0.163639,-0.011901,...,-0.028399,-0.062131,-0.999983,-0.225753,-0.221161,0.009844,0.054524,-0.119893,0.529306,-0.202766,0.143812,-0.047916,-0.212078,0.053727,0.999998,-0.995530,-0.033023,-0.011080,-0.173867,-0.044845,-0.966912,-0.993743,-0.077215,0.170830,-0.037026,0.091869,0.999614,-0.998443,0.691898,0.144321,-0.370477,0.413160,-0.757180,-0.167172,0.001184,-0.240944,-0.903791,0.890541,0.539551,-0.060881
2,0.994161,0.321198,-0.816173,0.919914,0.384275,0.540132,0.058804,-0.013137,0.885211,-0.396692,-0.132781,0.974505,-0.983142,0.063994,-0.155593,-0.068041,-0.032636,-0.120653,-0.218149,-0.244029,-0.825965,0.999340,0.999980,0.793501,-0.350607,0.999993,0.906058,0.188085,0.126897,-0.144923,-0.007987,0.183489,-0.055392,-0.323950,0.002088,0.999986,0.005737,-0.066776,0.259358,-0.345393,...,0.269924,-0.094626,-0.999658,-0.419525,-0.061353,-0.192084,0.097201,0.263564,0.349031,-0.125898,0.409266,0.179921,-0.094777,0.335740,0.999984,-0.995990,-0.070235,0.342876,0.152105,-0.120379,-0.835088,-0.987900,0.561285,-0.556340,-0.080919,0.031692,0.999371,-0.964137,-0.249547,-0.280774,-0.339678,0.233878,-0.159580,0.084612,0.582453,-0.298648,-0.947983,-0.787903,0.088152,0.061869
3,0.999371,0.219726,-0.997043,0.915737,0.938324,0.354740,-0.957634,-0.268375,0.999538,-0.217264,0.056975,0.988587,-0.999717,-0.187608,-0.343832,-0.242380,-0.182141,-0.213708,-0.082080,-0.993839,-0.621489,0.998463,0.999993,0.993565,0.274817,0.999999,0.992998,-0.145593,0.121217,0.110130,0.168326,0.203767,-0.052538,-0.374700,0.117248,0.999992,-0.117724,0.057206,0.186188,0.070009,...,0.144663,0.153909,-0.999984,0.014434,-0.201309,0.103748,0.454853,-0.148085,0.231351,-0.135903,0.319238,-0.353717,-0.275320,0.300979,0.999997,-0.995446,0.079092,0.097596,-0.081836,0.125831,-0.993797,-0.999350,0.250811,0.577524,0.291716,0.015052,0.999796,-0.997346,0.957175,0.029973,-0.373495,0.207088,-0.736161,-0.228284,0.186568,-0.198086,-0.998501,0.974117,0.696353,0.042173
4,0.993550,0.305345,-0.912007,0.907005,0.632525,0.571457,-0.423483,-0.126026,0.955551,-0.438697,-0.118068,0.962295,-0.991873,0.034294,-0.167381,-0.028453,-0.070761,-0.309965,-0.215995,-0.594724,-0.621415,0.998913,0.999979,0.859973,0.037916,0.999994,0.941473,0.270936,0.262307,-0.129448,-0.058142,0.169341,-0.095812,-0.230890,0.017832,0.999986,-0.088987,-0.143221,0.230168,-0.249851,...,0.291229,0.049619,-0.999662,-0.472734,-0.055766,-0.153096,0.162398,0.231830,0.463658,-0.015155,0.258997,0.189534,-0.151412,0.261290,0.999989,-0.994307,-0.032350,0.385127,0.029546,-0.164645,-0.867400,-0.993389,0.589913,-0.205017,-0.041609,-0.014329,0.999375,-0.982083,0.273481,-0.3679

In [ ]:
type(features)

pandas.core.frame.DataFrame